In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() # 데이터 프레임의 첫번째행부터 5개 추출

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
# pandas unique : 명시된 컬럼에 중복을 제거한다.
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [7]:
# Species : 타겟 데이터( 정답 데이터 ) 그 외 - 학습 데이터
import numpy as np
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [8]:
fish_target = fish['Species'].to_numpy() # 타겟 데이터( 정답 데이터 )

In [9]:
from sklearn.model_selection import train_test_split # 학습 세트, 테스트 세트 분리

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target
)

In [10]:
# 데이터 전처리 - 표준 점수 변환
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input) # 훈련 데이터를 표준 점수로 변환.
test_scaled = ss.transform(test_input) # 테스트 데이터를 표준 점수로 변환
print(train_scaled[:5])

[[-0.53466354 -0.4585021  -0.54812353 -0.48509955 -0.48594611]
 [-0.93102429 -0.8486469  -0.95562062 -1.02677612 -1.06786586]
 [-1.14336041 -1.64844374 -1.67100439 -1.72943671 -1.9784999 ]
 [ 0.24390223  0.2242513   0.42081398  1.24904944  0.1820072 ]
 [-0.97632266 -1.02421206 -1.01900905 -0.97378442 -1.03745428]]


In [30]:
# k-최근접 이웃 분류 - 분류, 확률 예측
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target) # 학습

print(kn.score(train_scaled, train_target)) # 훈련 세트 정확도
print(kn.score(test_scaled, test_target)) # 테스트 세트 정확도

0.8739495798319328
0.75


In [31]:
# 클래스 목록 (분류 항목) - 알파벳순서대로
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [32]:
print(kn.predict(test_scaled[:5]))

['Bream' 'Bream' 'Smelt' 'Smelt' 'Perch']


In [33]:
# 분류 기준 확률 kn.predict_probo(..)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.667 0.333 0.    0.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]]


In [34]:
distances, indexes = kn.kneighbors(test_scaled[2:3])
print(train_target[indexes]) # 확률로 보기에는 아직 어색하다.

[['Smelt' 'Smelt' 'Smelt']]


In [36]:
# 로지스틱 회귀 - 이진 분리한(도미, 광어) 데이터 가져오기.
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
print(target_bream_smelt[:5])

['Smelt' 'Bream' 'Bream' 'Bream' 'Smelt']


In [37]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt) # 훈련!

LogisticRegression()

In [38]:
print(lr.predict(train_bream_smelt[:5]))

['Smelt' 'Bream' 'Bream' 'Bream' 'Smelt']


In [39]:
# 확률 predict_proba
print(np.round(lr.predict_proba(train_bream_smelt[:5]), decimals=3)) 

[[0.043 0.957]
 [0.988 0.012]
 [0.995 0.005]
 [0.995 0.005]
 [0.027 0.973]]


In [40]:
# 1 - 양성 클래스 : 방어, 0 - 음성 클래스 : 방어
print(lr.classes_) #1. 양성 클래스 기준으로 구하면서 형재의 기준은 방어, 0 - 음성 클래스 - 방어 로 구하게 됀다.

['Bream' 'Smelt']


In [42]:
# 가중치(기울기)와 절편
print(lr.coef_, lr.intercept_)

[[-0.40383828 -0.58209297 -0.66565062 -0.97831134 -0.73635774]] [-2.57966999]


In [43]:
# 학습을 통해서 구한 가중치와 절편으로 실제 방정식의 결과 값을 구하는 함수 decision_function
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions) # 결과값을 0 ~ 1 사이로 변환 - 시그모이드 함수

[ 3.1027266  -4.44479889 -5.30337253 -5.24293346  3.57898705]


In [45]:
# 시그모이드 함수로 값을 0 ~ 1 사이로 변환 작업 시작.
from scipy.special import expit # 시그모이드 함수
print(np.round(expit(decisions), decimals=3))

[0.957 0.012 0.005 0.005 0.973]


In [49]:
# 로지스틱 회귀를 통한 다중 분류 / 확률 - 소프트맥스 함수
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target) # 학습 : 타겟은 정답데이터, scaled는 학습데이터

print(lr.score(train_scaled, train_target)) # 학습 세트
print(lr.score(test_scaled, test_target)) # 테스트 세트

0.9411764705882353
0.925


In [50]:
print(lr.predict(test_scaled[:5]))

['Bream' 'Bream' 'Smelt' 'Smelt' 'Roach']


In [51]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [52]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.553 0.36  0.    0.    0.024 0.    0.062]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.003 0.033 0.    0.006 0.957 0.   ]
 [0.    0.005 0.054 0.    0.009 0.932 0.   ]
 [0.    0.023 0.225 0.003 0.721 0.005 0.023]]


In [53]:
print(lr.coef_, lr.intercept_)

[[-1.61342293 -1.13621894  2.1983018   7.85641517 -0.74059617]
 [-0.57265803 -1.96669239 -3.21784679  6.42465664 -2.08618054]
 [ 3.32628731  6.87465984 -9.0126055  -6.48377087  4.65157228]
 [ 0.0489308   3.68564673  3.58969465 -3.02228264 -2.15130009]
 [-2.22291753 -5.78002303  5.43275331 -1.61339572  2.63068511]
 [-0.89109557  1.0685594   1.32957033 -4.28239045 -5.02752591]
 [ 1.92487596 -2.74593163 -0.3198678   1.12076786  2.72334531]] [ 0.58731019 -0.47533767  2.63863889 -0.40224688  2.21139338 -7.1569058
  2.5971479 ]


In [55]:
# decision_function을 통해서 각 클래스별 z값을 구할 수 있다
decisions = lr.decision_function(test_scaled[:5])
print(np.round(decisions, decimals=3))

[[  5.059   4.63   -2.295  -3.397   1.911  -8.785   2.877]
 [ 16.527   3.964  -4.063  -1.107  -1.306 -22.247   8.233]
 [-11.436   1.966   4.237  -2.894   2.599   7.595  -2.066]
 [-11.07    1.937   4.296  -2.843   2.516   7.153  -1.988]
 [ -4.305   0.356   2.653  -1.816   3.819  -1.088   0.381]]


In [56]:
from scipy.special import softmax

proba = softmax(decisions, axis=1) # 행별로(axis=1)
print(np.round(proba,decimals=3))

[[0.553 0.36  0.    0.    0.024 0.    0.062]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.003 0.033 0.    0.006 0.957 0.   ]
 [0.    0.005 0.054 0.    0.009 0.932 0.   ]
 [0.    0.023 0.225 0.003 0.721 0.005 0.023]]
